In [ ]:
# TO GRAPH STUFFS, NOT SO INTERESTINGS


library(igraph)
library(repr)
source("func.r")

In [ ]:
get_expected_func <- function(graph, observable, param) {
    if (observable == "m") {
        Tc <- get_Tc(graph)

        param <- graph$param
        if (graph$nickname == "ba"|| graph$nickname == "sf") {
            if (param > 2 && param < 3) {
                return(function(t, a, off) (off + a * (t)^(-1 / (3 - param))))
            } else if (param == 3) {
                return(function(t, a, off) (off + a * exp(-2 * t / get_avg_degree(graph))))
                # return(function(t, a, off) (off + a * exp(-t / 2)))
            } else if (param > 3 && param < 5) {
                return(function(t, a, off) {
                    tau <- 1 - t / Tc
                    return(off + a * tau^(1 / (param - 3)))
                })
            } else if (param == 5) {
                return(function(t, a, off) {
                    tau <- 1 - t / Tc
                    off + a * (tau^(1 / 2)) / ((log(tau^-1))^(1 / 2)) # check if the ^-1 is the arg or the log
                })
            } else if (param > 5) {
                return(function(t, a, off) {
                    tau <- 1 - t / Tc
                    off + a * tau^(1 / 2)
                })
            } else {
                cat("Parametro non valido: ", param, " per il modello di Barabasi Albert (scalefree)\n")
            }
        }
    } else if (observable == "sus") {
        Tc <- get_Tc(graph)
        return(function(t, a, off) {
            if (graph$nickname == "ba"|| graph$nickname == "sf") {
                if (param > 2 && param <= 3) {
                    y <- off + a * t^-1
                } else if (param > 3) {
                    tau <- abs(1 - t / Tc)
                    y <- off + a * tau^-1
                } else {
                    cat("Parametro non valido: ", param, " per il modello di Barabasi Albert (scalefree)\n")
                }
            }
            return(y)
        })
    } else if (observable == "heat") {
        Tc <- get_Tc(graph)
        return(function(t, a, off) {
            if (graph$nickname == "ba" || graph$nickname == "sf") {
                if (param > 2 && param < 3) {
                    y <- off + a * t^(-(param - 1) / (3 - param))
                } else if (param == 3) {
                    y <- off + a * t^2 * exp(-4 * t / get_avg_degree(graph))
                } else if (param > 3 && param < 5) {
                    tau <- 1 - t / Tc
                    # cat(tau, "\n")
                    y <- off + a * tau^((5 - param) / (param - 3))
                    # cat(off + a * tau^((5 - param) / (param - 3)), "\n")
                } else if (param == 5) {
                    tau <- 1 - t / Tc
                    y <- 1 / log(tau^-1) # <------------------- check
                } else if (param > 5) {
                    cat("param > 5 imposto come funzione l'identità\n")
                    y <- t
                } else {
                    cat("Parametro non valido: ", param, " per il modello di Barabasi Albert (scalefree)\n")
                }
            }
            return(y)
        })
    }
}

calculate_fit <- function(graph, observable, param, datax, datay, Tc) {
    fitfunc <- get_expected_func(graph, observable, param)

    if (observable == "m") {
        if (param > 3) {
            index <- datax < Tc
        } else {
            index <- 1:length(datax)
        }
        datax <- datax[index]
        datay <- datay[index]

        dfm <- data.frame(datax, datay)
        m <- nls(datay ~ fitfunc(datax, a, off),
            data = dfm, start = list(a = 1, off = 0), trace = F, algorithm = "port", lower = c(0.001, -10), upper = c(20, 10)
        )


        return(coef(m))
    } else if (observable == "sus") {
        index <- datax != Tc
        datax <- datax[index]
        datay <- datay[index]

        # datay <- abs(datay)

        dfm <- data.frame(datax, datay)
        m <- nls(datay ~ fitfunc(datax, a, off),
            data = dfm, start = list(a = 1, off = 0), trace = F, algorithm = "port",
            lower = c(0.001, -10), upper = c(20, 10)
        )
        return(coef(m))
    } else if (observable == "heat") {
        index <- datax < Tc
        datax <- datax[index]
        datay <- datay[index]
        # datay <- abs(datay)

        dfm <- data.frame(datax, datay)
        m <- nls(datay ~ fitfunc(datax, a, off),
            data = dfm, start = list(a = 1, off = 0), trace = F, algorithm = "port",
            lower = c(0.001, -10), upper = c(20, 10)
        )
        return(coef(m))
    }
}


In [ ]:
filenames <- c(
    # "Nba2.5_N3000_Step30000_2023_10_08_02_41_34.RData",
    # "Nba3_N3000_Step30000_2023_10_08_04_13_57.RData",
    "Nba3.5_N3000_Step30000_2023_10_08_05_36_56.RData",
    "Nba4_N3000_Step30000_2023_10_08_06_44_44.RData",
    "Nba4.5_N3000_Step30000_2023_10_08_07_48_21.RData",
    "Nba5_N3000_Step30000_2023_10_08_08_44_38.RData",
    "Nba7_N3000_Step30000_2023_10_08_09_34_47.RData"
)
# filenames <- c(
#     # "Nsf2.5_N800_Step30000_2023_10_09_09_17_49.RData",
#     "Nsf3.5_N800_Step30000_2023_10_09_02_30_08.RData",
#     "Nsf4_N800_Step30000_2023_10_09_01_37_26.RData",
#     "Nsf4.5_N800_Step30000_2023_10_09_00_42_53.RData"
#     # "Nsf5_N800_Step30000_2023_10_08_23_58_35.RData",
#     # "Nsf6_N800_Step30000_2023_10_08_23_21_46.RData"
# )
# colors <- c("blue3", "red", "darkgreen", "magenta4", "darkorange", "chartreuse1")
TOTF <- length(filenames)
if (TOTF > 5) {
    colors <- c("red", "darkorange1", "gold1", "chartreuse1", "darkgreen", "dodgerblue", "blue")
} else {
    colors <- c("red", "darkorange1", "darkgreen", "dodgerblue", "blue")    
}


observable <- "M"
PDF <- F

if (PDF) {
    pdf(paste(observable, "3000.pdf", sep = ""), width = 12, height = 7)
} else {
    options(repr.plot.width = 16, repr.plot.height = 8)
}
# pdf(paste(observable, "3000fit.pdf", sep = ""), width = 16, height = 8)

title <- ""
if (observable == "M") {
    title <- "Magnetization"
} else if (observable == "S") {
    title <- "Susceptibility"
} else if (observable == "H") {
    title <- "Specific Heat"
}

maxy <- ifelse(observable == "M", 1, 3)
plot(c(), c(),
    main = paste(title, "for scale-free networks (N=3000)"), xlab = "Kb * T (Kb=1)", ylab = observable, col = "blue", pch = 19,
    # xlim = c(0, max(kbts)), ylim=c(0, max(c(datay_m, datay_heat, datay_sus)))
    xlim = c(0.25, 5), ylim = c(0, 1)
)

legend_texts <- rep("", length(filenames))
for (i in 1:length(filenames)) {
    load(filenames[i])
    N <- graph$N

    Tc <- get_Tc(graph)
    xc <- seq(Tc - 1, Tc - 0.01, 0.01)
    xfit <- seq(0.1, max(kbts), 0.01)

    mult_sus <- 1
    mult_heat <- 1

    filt <- !(is.na(graph$avgs_m) | is.na(graph$avg_sus) | is.na(graph$avg_heat))
    tc_index <- match(TRUE, abs(kbts - Tc) < 0.0001)
    filt <- filt & c((1:tc_index) %% 4 == 0, rep(T, length(kbts) - tc_index))
    filt <- filt & (kbts < Tc)

    kbts <- kbts[filt]
    datay_m <- (abs(graph$avgs_m) / N)[filt]
    datay_sus <- graph$avg_sus[filt] * mult_sus
    datay_heat <- graph$avg_heat[filt] * mult_heat

    if (observable == "M") {
        lines(kbts, datay_m, col = colors[i], pch = 19)
        points(kbts, datay_m, col = colors[i], pch = 19)
    } else if (observable == "S") {
        lines(kbts, datay_sus, col = colors[i], pch = 19)
        points(kbts, datay_sus, col = colors[i], pch = 19)
    } else if (observable == "H") {
        lines(kbts, datay_heat, col = colors[i], pch = 19)
        points(kbts, datay_heat, col = colors[i], pch = 19)
    }

    # points(kbts, datay_heat, col = "red", pch = 17)

    coefs_m <- calculate_fit(graph, "m", graph$param, kbts, datay_m, Tc)
    fitfunc_m <- get_expected_func(graph, "m", graph$param)
    lines(xfit[xfit < Tc], do.call(fitfunc_m, append(list(xfit[xfit < Tc]), coefs_m)), lwd = 2, col = colors[i])

    # coefs_sus <- calculate_fit(graph, "sus", graph$param, kbts, datay_sus, Tc)
    # fitfunc_sus <- get_expected_func(graph, "sus", graph$param)
    # lines(xfit[xfit < Tc], do.call(fitfunc_sus, append(list(xfit[xfit < Tc]), coefs_sus)), lwd = 2, col = "darkgreen")

    # coefs_heat <- calculate_fit(graph, "heat", graph$param, kbts, datay_heat, Tc)
    # fitfunc_heat <- get_expected_func(graph, "heat", graph$param)
    # lines(xfit, do.call(fitfunc_heat, append(list(xfit), coefs_heat)), lwd = 2, col = "red")

    # lines(xc, ym, lwd = 2, col = "blue")
    # lines(xc, ysus * 0.1, lwd = 2, col = "darkgreen")
    # lines(xc, yheat * 0.15, lwd = 2, col = "red")
    abline(v = Tc, col = colors[i], lwd = 2, lty = 2)
    # legend(
    #     # x = "topright",
    #     x = 5.5, y = 1,
    #     legend = c(
    #         "Magnetization M ",
    #         paste("Susceptibility * ", mult_sus, "S"),
    #         paste("Specific Heat *", mult_heat, "H"),
    #         "Theoretical Critical Temperature Tc",
    #         "Expected Critical Behavior of M around Tc", "Expected Critical Behavior of S around Tc", "Expected Critical Behavior of H around Tc"
    #     ),
    #     col = c("blue", "darkgreen", "red", "black", "blue", "darkgreen", "red"),
    #     pch = c(19, 15, 17, NA, NA, NA, NA, NA), lwd = c(0, 0, 0, 2, 2, 2, 2, 2), lty = c(0, 0, 0, 1, 1, 1, 1, 1)
    # )
    legend_texts[i] <- paste("gamma = ", graph$param)
}

legend(
    # x = "topright",
    x = 2, y =2,
    legend = c(legend_texts, "Critical Temperature Tc"),
    col = c(colors[1:length(filenames)], "black"),
    pch = c(rep(19, length(filenames)), NA),
    lwd = c(rep(1, length(filenames)), 1.5),
    lty = c(rep(1, length(filenames)), 2),
    bg = "white"
)

if (PDF) {
    dev.off()
}


In [ ]:
my_vector <- c(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

# Index k (change this to your desired index)
k <- 3

# Select every other element after index k
after_k <- my_vector[(k + 1):length(my_vector) %% 2 == 0]
before_k <- my_vector[1:k]
final_vector <- c(before_k, after_k)

In [ ]:
tc_index = 3
indexes = c(rep(T, k), ((k + 1):length(my_vector) )%% 2 == 0)
indexes

In [ ]:
Tc
kbts


In [ ]:
 tc_index <- match(TRUE, abs(kbts - Tc) < 0.0001)
 tc_index
# filt <- c((1:tc_index) %% 2 == 0, rep(T, length(kbts) - tc_index))

In [ ]:
filt <- c((1:tc_index) %% 2 == 0, rep(T, length(kbts) - tc_index))
filt

In [ ]:
########################### ERDOSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS

filenames <- c(
    "Ner1_N1000_Step30000_2023_10_09_19_47_45.RData"
)


TOTF <- length(filenames)
if (TOTF > 5) {
    colors <- c("red", "darkorange1", "gold1", "chartreuse1", "darkgreen", "dodgerblue", "blue")
} else {
    colors <- c("red", "darkorange1", "darkgreen", "dodgerblue", "blue")    
}


observable <- "M"
PDF <- F

if (PDF) {
    pdf(paste(observable, "3000.pdf", sep = ""), width = 12, height = 7)
} else {
    options(repr.plot.width = 16, repr.plot.height = 8)
}
# pdf(paste(observable, "3000fit.pdf", sep = ""), width = 16, height = 8)

title <- ""
if (observable == "M") {
    title <- "Magnetization"
} else if (observable == "S") {
    title <- "Susceptibility"
} else if (observable == "H") {
    title <- "Specific Heat"
}

maxy <- ifelse(observable == "M", 1, 3)
plot(c(), c(),
    main = paste(title, "for scale-free networks (N=3000)"), xlab = "Kb * T (Kb=1)", ylab = observable, col = "blue", pch = 19,
    # xlim = c(0, max(kbts)), ylim=c(0, max(c(datay_m, datay_heat, datay_sus)))
    xlim = c(0.25, 5), ylim = c(0, 1)
)

legend_texts <- rep("", length(filenames))
for (i in 1:length(filenames)) {
    load(filenames[i])
    N <- graph$N

    Tc <- get_Tc(graph)
    xc <- seq(Tc - 1, Tc - 0.01, 0.01)
    xfit <- seq(0.1, max(kbts), 0.01)

    mult_sus <- 1
    mult_heat <- 1

    filt <- !(is.na(graph$avgs_m) | is.na(graph$avg_sus) | is.na(graph$avg_heat))
    tc_index <- match(TRUE, abs(kbts - Tc) < 0.0001)
    # filt <- filt & c((1:tc_index) %% 4 == 0, rep(T, length(kbts) - tc_index))
    # filt <- filt & (kbts < Tc)

    kbts <- kbts[filt]
    datay_m <- (abs(graph$avgs_m) / N)[filt]
    datay_sus <- graph$avg_sus[filt] * mult_sus
    datay_heat <- graph$avg_heat[filt] * mult_heat

    if (observable == "M") {
        lines(kbts, datay_m, col = colors[i], pch = 19)
        points(kbts, datay_m, col = colors[i], pch = 19)
    } else if (observable == "S") {
        lines(kbts, datay_sus, col = colors[i], pch = 19)
        points(kbts, datay_sus, col = colors[i], pch = 19)
    } else if (observable == "H") {
        lines(kbts, datay_heat, col = colors[i], pch = 19)
        points(kbts, datay_heat, col = colors[i], pch = 19)
    }

    # points(kbts, datay_heat, col = "red", pch = 17)

    # coefs_m <- calculate_fit(graph, "m", graph$param, kbts, datay_m, Tc)
    # fitfunc_m <- get_expected_func(graph, "m", graph$param)
    # lines(xfit[xfit < Tc], do.call(fitfunc_m, append(list(xfit[xfit < Tc]), coefs_m)), lwd = 2, col = colors[i])

    # coefs_sus <- calculate_fit(graph, "sus", graph$param, kbts, datay_sus, Tc)
    # fitfunc_sus <- get_expected_func(graph, "sus", graph$param)
    # lines(xfit[xfit < Tc], do.call(fitfunc_sus, append(list(xfit[xfit < Tc]), coefs_sus)), lwd = 2, col = "darkgreen")

    # coefs_heat <- calculate_fit(graph, "heat", graph$param, kbts, datay_heat, Tc)
    # fitfunc_heat <- get_expected_func(graph, "heat", graph$param)
    # lines(xfit, do.call(fitfunc_heat, append(list(xfit), coefs_heat)), lwd = 2, col = "red")

    # lines(xc, ym, lwd = 2, col = "blue")
    # lines(xc, ysus * 0.1, lwd = 2, col = "darkgreen")
    # lines(xc, yheat * 0.15, lwd = 2, col = "red")
    abline(v = Tc, col = colors[i], lwd = 2, lty = 2)
    # legend(
    #     # x = "topright",
    #     x = 5.5, y = 1,
    #     legend = c(
    #         "Magnetization M ",
    #         paste("Susceptibility * ", mult_sus, "S"),
    #         paste("Specific Heat *", mult_heat, "H"),
    #         "Theoretical Critical Temperature Tc",
    #         "Expected Critical Behavior of M around Tc", "Expected Critical Behavior of S around Tc", "Expected Critical Behavior of H around Tc"
    #     ),
    #     col = c("blue", "darkgreen", "red", "black", "blue", "darkgreen", "red"),
    #     pch = c(19, 15, 17, NA, NA, NA, NA, NA), lwd = c(0, 0, 0, 2, 2, 2, 2, 2), lty = c(0, 0, 0, 1, 1, 1, 1, 1)
    # )
    legend_texts[i] <- paste("gamma = ", graph$param)
}

legend(
    # x = "topright",
    x = 2, y =2,
    legend = c(legend_texts, "Critical Temperature Tc"),
    col = c(colors[1:length(filenames)], "black"),
    pch = c(rep(19, length(filenames)), NA),
    lwd = c(rep(1, length(filenames)), 1.5),
    lty = c(rep(1, length(filenames)), 2),
    bg = "white"
)

if (PDF) {
    dev.off()
}